In [1]:
from obspy import read, Stream, UTCDateTime
import numpy as np
from mudpy.hfsims import windowed_gaussian, apply_spectrum
from mudpy.forward import gnss_psd
import matplotlib.pyplot as plt
import h5py
from glob import glob

In [2]:
project_name = 'july6'
GF_list = 'rc_gflist.gflist' # Station file used in fq gen code
vel_mod = 'mojave'
stas_name = GF_list.split('.')[0]
fq_dir = '/hdd/rc_fq/summer23/' + project_name + '/'
arrival_save_dir = '/hdd/rc_fq/summer23/' + project_name + '_fq_parrivals/'

ruptures = np.load('/hdd/rc_fq/summer23/' + project_name + '_ruptures.npy')
test_ruptures = ['july6.000000']

stas = np.load('/hdd/rc_fq/summer23/' + stas_name + '_station_names.npy')
test_stas = ['P595', 'P594', 'CCCC']
# print(stas)

In [3]:
data_list = []

mag_list = []
sta_list = []
rupt_list = []

counter = 0

for rupt in ruptures:
# for krupt in range(len(test_rupts)):
    
    counter += 1
    print('Rupture ' + rupt + ' (' + str(counter) + '/' + str(len(ruptures)) + ')')
    
    log = glob(fq_dir + 'output/ruptures/' + rupt + '.log')
    log = open(log[0],'r')
    line = log.readlines()
    
    mag = str(line[15][21:27])
#     print(mag)
    
    for sta in stas:
        
#         print(sta)
        arrivals = np.genfromtxt(arrival_save_dir + stas_name + '_arrival_times_' + rupt + '.csv', dtype = 'U')
#         print(arrivals)
            
        # Read in data 
        
        stN = read(fq_dir + 'output/waveforms/' + rupt + '/' + sta + '.LYN.sac') 
        stE = read(fq_dir + 'output/waveforms/' + rupt + '/' + sta + '.LYE.sac') 
        stZ = read(fq_dir + 'output/waveforms/' + rupt + '/' + sta + '.LYZ.sac') 
        
#         print(stN[0].stats.starttime)
        
#         stN.plot()
        
        N_data = stN[0].data
        E_data = stE[0].data
        Z_data = stZ[0].data
        
        ### Zero-pad this data ###
        
        N_data_padded = np.pad(N_data, 128, mode = 'constant')
        E_data_padded = np.pad(E_data, 128, mode = 'constant')
        Z_data_padded = np.pad(Z_data, 128, mode = 'constant')
        
        stN_pad = stN.copy()
        stN_pad[0].data = N_data_padded
        
        stE_pad = stE.copy()
        stE_pad[0].data = E_data_padded
        
        stZ_pad = stZ.copy()
        stZ_pad[0].data = Z_data_padded
        
#         stN.plot()
#         stN_pad.plot()
#         print(stN_pad[0].stats.starttime)
        
        npts = stN_pad[0].stats.npts

        ### Trim around the arrival time ###
        
        stas_arrival = arrivals[:,1] # Station list from arrivals file
        
        i = np.where(stas_arrival == sta)[0]
        arrival = arrivals[i,2][0]
#         print(arrival)

        # Grab the arrival time

        arr_time = UTCDateTime(arrival)
        arr_time = arr_time + 128 # To account for padding at front
        starttime = arr_time - 128
        endtime = arr_time + 127 # Needs to be 256 samples, not 256 seconds

#         arr_time = arr_time + 3 # To account for padding at front
#         starttime = arr_time - 3
#         endtime = arr_time + 3
        
#         print(starttime)
#         print(arr_time)
#         print(endtime)
        
        stN_trim = stN_pad.trim(starttime, endtime)
        stE_trim = stE_pad.trim(starttime, endtime)
        stZ_trim = stZ_pad.trim(starttime, endtime)
        
        stN_trim_data = stN_trim[0].data # Middle sample is the (npts + 1)/2 sample, index is (npts + 1)/2 - 1
        stE_trim_data = stE_trim[0].data
        stZ_trim_data = stZ_trim[0].data
        
#         print(stN_trim[0].stats.starttime)
#         print(stN_trim[0].stats.endtime)
#         print(stN_trim[0].stats.npts) # Middle sample is the (npts + 1)/2 sample, index is (npts + 1)/2 - 1
#         print(len(stN_trim_data))
#         stN_trim[0].plot()

        npts = stN_trim[0].stats.npts
#         print(npts)
        arrival_idx = int((npts + 1)/2 - 1)
        
        pick_N = stN_trim_data[arrival_idx]
        pick_E = stE_trim_data[arrival_idx]   
        pick_Z = stZ_trim_data[arrival_idx]
        
#         print(pick_N)

        stN_demean = stN_trim_data - pick_N
        stE_demean = stE_trim_data - pick_E
        stZ_demean = stZ_trim_data - pick_Z    
        
        stN_zeroed = stN_demean
        stN_zeroed[0:128] = 0 # Remember that the stop index is excluded
        
        stE_zeroed = stE_demean
        stE_zeroed[0:128] = 0 
        
        stZ_zeroed = stZ_demean
        stZ_zeroed[0:128] = 0 
        
        
        ### Combine N, E, and Z components into one array ###
        
        comb_data = np.append(stN_zeroed, stE_zeroed)
        comb_data = np.append(comb_data, stZ_zeroed) # Order: N, E, Z
        
        ### Stream-normalize ###
        
        combdata_norm = comb_data / np.max(abs(comb_data))
        
#         print(comb_data.shape)
        
        # N: indices 0 through 255 pick at index 128)
        # E: indices 256 through 511 (pick at index 384)
        # Z: indices 512 through 767 (pick at index 640)
        
#         plt.plot(comb_data)
#         plt.show()
        
#         plt.plot(combdata_norm) # Checking to make sure everything lines up
#         plt.show()

        ### Adding new data to an array - each row = new station ### 
    
        data_list.append(combdata_norm) # Add clean data instead
        
        ### Add magnitude to list
        
        rupt_list.append(str(rupt))
        sta_list.append(str(sta))
        mag_list.append(str(mag))
        

Rupture july6.000000 (1/3300)
Rupture july6.000001 (2/3300)
Rupture july6.000002 (3/3300)
Rupture july6.000003 (4/3300)
Rupture july6.000004 (5/3300)
Rupture july6.000005 (6/3300)
Rupture july6.000006 (7/3300)
Rupture july6.000007 (8/3300)
Rupture july6.000008 (9/3300)
Rupture july6.000009 (10/3300)
Rupture july6.000010 (11/3300)
Rupture july6.000011 (12/3300)
Rupture july6.000012 (13/3300)
Rupture july6.000013 (14/3300)
Rupture july6.000014 (15/3300)
Rupture july6.000015 (16/3300)
Rupture july6.000016 (17/3300)
Rupture july6.000017 (18/3300)
Rupture july6.000018 (19/3300)
Rupture july6.000019 (20/3300)
Rupture july6.000020 (21/3300)
Rupture july6.000021 (22/3300)
Rupture july6.000022 (23/3300)
Rupture july6.000023 (24/3300)
Rupture july6.000024 (25/3300)
Rupture july6.000025 (26/3300)
Rupture july6.000026 (27/3300)
Rupture july6.000027 (28/3300)
Rupture july6.000028 (29/3300)
Rupture july6.000029 (30/3300)
Rupture july6.000030 (31/3300)
Rupture july6.000031 (32/3300)
Rupture july6.000

Rupture july6.000260 (261/3300)
Rupture july6.000261 (262/3300)
Rupture july6.000262 (263/3300)
Rupture july6.000263 (264/3300)
Rupture july6.000264 (265/3300)
Rupture july6.000265 (266/3300)
Rupture july6.000266 (267/3300)
Rupture july6.000267 (268/3300)
Rupture july6.000268 (269/3300)
Rupture july6.000269 (270/3300)
Rupture july6.000270 (271/3300)
Rupture july6.000271 (272/3300)
Rupture july6.000272 (273/3300)
Rupture july6.000273 (274/3300)
Rupture july6.000274 (275/3300)
Rupture july6.000275 (276/3300)
Rupture july6.000276 (277/3300)
Rupture july6.000277 (278/3300)
Rupture july6.000278 (279/3300)
Rupture july6.000279 (280/3300)
Rupture july6.000280 (281/3300)
Rupture july6.000281 (282/3300)
Rupture july6.000282 (283/3300)
Rupture july6.000283 (284/3300)
Rupture july6.000284 (285/3300)
Rupture july6.000285 (286/3300)
Rupture july6.000286 (287/3300)
Rupture july6.000287 (288/3300)
Rupture july6.000288 (289/3300)
Rupture july6.000289 (290/3300)
Rupture july6.000290 (291/3300)
Rupture 

Rupture july6.000517 (518/3300)
Rupture july6.000518 (519/3300)
Rupture july6.000519 (520/3300)
Rupture july6.000520 (521/3300)
Rupture july6.000521 (522/3300)
Rupture july6.000522 (523/3300)
Rupture july6.000523 (524/3300)
Rupture july6.000524 (525/3300)
Rupture july6.000525 (526/3300)
Rupture july6.000526 (527/3300)
Rupture july6.000527 (528/3300)
Rupture july6.000528 (529/3300)
Rupture july6.000529 (530/3300)
Rupture july6.000530 (531/3300)
Rupture july6.000531 (532/3300)
Rupture july6.000532 (533/3300)
Rupture july6.000533 (534/3300)
Rupture july6.000534 (535/3300)
Rupture july6.000535 (536/3300)
Rupture july6.000536 (537/3300)
Rupture july6.000537 (538/3300)
Rupture july6.000538 (539/3300)
Rupture july6.000539 (540/3300)
Rupture july6.000540 (541/3300)
Rupture july6.000541 (542/3300)
Rupture july6.000542 (543/3300)
Rupture july6.000543 (544/3300)
Rupture july6.000544 (545/3300)
Rupture july6.000545 (546/3300)
Rupture july6.000546 (547/3300)
Rupture july6.000547 (548/3300)
Rupture 

Rupture july6.000774 (775/3300)
Rupture july6.000775 (776/3300)
Rupture july6.000776 (777/3300)
Rupture july6.000777 (778/3300)
Rupture july6.000778 (779/3300)
Rupture july6.000779 (780/3300)
Rupture july6.000780 (781/3300)
Rupture july6.000781 (782/3300)
Rupture july6.000782 (783/3300)
Rupture july6.000783 (784/3300)
Rupture july6.000784 (785/3300)
Rupture july6.000785 (786/3300)
Rupture july6.000786 (787/3300)
Rupture july6.000787 (788/3300)
Rupture july6.000788 (789/3300)
Rupture july6.000789 (790/3300)
Rupture july6.000790 (791/3300)
Rupture july6.000791 (792/3300)
Rupture july6.000792 (793/3300)
Rupture july6.000793 (794/3300)
Rupture july6.000794 (795/3300)
Rupture july6.000795 (796/3300)
Rupture july6.000796 (797/3300)
Rupture july6.000797 (798/3300)
Rupture july6.000798 (799/3300)
Rupture july6.000799 (800/3300)
Rupture july6.000800 (801/3300)
Rupture july6.000801 (802/3300)
Rupture july6.000802 (803/3300)
Rupture july6.000803 (804/3300)
Rupture july6.000804 (805/3300)
Rupture 

Rupture july6.001030 (1031/3300)
Rupture july6.001031 (1032/3300)
Rupture july6.001032 (1033/3300)
Rupture july6.001033 (1034/3300)
Rupture july6.001034 (1035/3300)
Rupture july6.001035 (1036/3300)
Rupture july6.001036 (1037/3300)
Rupture july6.001037 (1038/3300)
Rupture july6.001038 (1039/3300)
Rupture july6.001039 (1040/3300)
Rupture july6.001040 (1041/3300)
Rupture july6.001041 (1042/3300)
Rupture july6.001042 (1043/3300)
Rupture july6.001043 (1044/3300)
Rupture july6.001044 (1045/3300)
Rupture july6.001045 (1046/3300)
Rupture july6.001046 (1047/3300)
Rupture july6.001047 (1048/3300)
Rupture july6.001048 (1049/3300)
Rupture july6.001049 (1050/3300)
Rupture july6.001050 (1051/3300)
Rupture july6.001051 (1052/3300)
Rupture july6.001052 (1053/3300)
Rupture july6.001053 (1054/3300)
Rupture july6.001054 (1055/3300)
Rupture july6.001055 (1056/3300)
Rupture july6.001056 (1057/3300)
Rupture july6.001057 (1058/3300)
Rupture july6.001058 (1059/3300)
Rupture july6.001059 (1060/3300)
Rupture ju

...

Rupture july6.002524 (2525/3300)
Rupture july6.002525 (2526/3300)
Rupture july6.002526 (2527/3300)
Rupture july6.002527 (2528/3300)
Rupture july6.002528 (2529/3300)
Rupture july6.002529 (2530/3300)
Rupture july6.002530 (2531/3300)
Rupture july6.002531 (2532/3300)
Rupture july6.002532 (2533/3300)
Rupture july6.002533 (2534/3300)
Rupture july6.002534 (2535/3300)
Rupture july6.002535 (2536/3300)
Rupture july6.002536 (2537/3300)
Rupture july6.002537 (2538/3300)
Rupture july6.002538 (2539/3300)
Rupture july6.002539 (2540/3300)
Rupture july6.002540 (2541/3300)
Rupture july6.002541 (2542/3300)
Rupture july6.002542 (2543/3300)
Rupture july6.002543 (2544/3300)
Rupture july6.002544 (2545/3300)
Rupture july6.002545 (2546/3300)
Rupture july6.002546 (2547/3300)
Rupture july6.002547 (2548/3300)
Rupture july6.002548 (2549/3300)
Rupture july6.002549 (2550/3300)
Rupture july6.002550 (2551/3300)
Rupture july6.002551 (2552/3300)
Rupture july6.002552 (2553/3300)
Rupture july6.002553 (2554/3300)
Rupture ju

Rupture july6.002773 (2774/3300)
Rupture july6.002774 (2775/3300)
Rupture july6.002775 (2776/3300)
Rupture july6.002776 (2777/3300)
Rupture july6.002777 (2778/3300)
Rupture july6.002778 (2779/3300)
Rupture july6.002779 (2780/3300)
Rupture july6.002780 (2781/3300)
Rupture july6.002781 (2782/3300)
Rupture july6.002782 (2783/3300)
Rupture july6.002783 (2784/3300)
Rupture july6.002784 (2785/3300)
Rupture july6.002785 (2786/3300)
Rupture july6.002786 (2787/3300)
Rupture july6.002787 (2788/3300)
Rupture july6.002788 (2789/3300)
Rupture july6.002789 (2790/3300)
Rupture july6.002790 (2791/3300)
Rupture july6.002791 (2792/3300)
Rupture july6.002792 (2793/3300)
Rupture july6.002793 (2794/3300)
Rupture july6.002794 (2795/3300)
Rupture july6.002795 (2796/3300)
Rupture july6.002796 (2797/3300)
Rupture july6.002797 (2798/3300)
Rupture july6.002798 (2799/3300)
Rupture july6.002799 (2800/3300)
Rupture july6.002800 (2801/3300)
Rupture july6.002801 (2802/3300)
Rupture july6.002802 (2803/3300)
Rupture ju

Rupture july6.003022 (3023/3300)
Rupture july6.003023 (3024/3300)
Rupture july6.003024 (3025/3300)
Rupture july6.003025 (3026/3300)
Rupture july6.003026 (3027/3300)
Rupture july6.003027 (3028/3300)
Rupture july6.003028 (3029/3300)
Rupture july6.003029 (3030/3300)
Rupture july6.003030 (3031/3300)
Rupture july6.003031 (3032/3300)
Rupture july6.003032 (3033/3300)
Rupture july6.003033 (3034/3300)
Rupture july6.003034 (3035/3300)
Rupture july6.003035 (3036/3300)
Rupture july6.003036 (3037/3300)
Rupture july6.003037 (3038/3300)
Rupture july6.003038 (3039/3300)
Rupture july6.003039 (3040/3300)
Rupture july6.003040 (3041/3300)
Rupture july6.003041 (3042/3300)
Rupture july6.003042 (3043/3300)
Rupture july6.003043 (3044/3300)
Rupture july6.003044 (3045/3300)
Rupture july6.003045 (3046/3300)
Rupture july6.003046 (3047/3300)
Rupture july6.003047 (3048/3300)
Rupture july6.003048 (3049/3300)
Rupture july6.003049 (3050/3300)
Rupture july6.003050 (3051/3300)
Rupture july6.003051 (3052/3300)
Rupture ju

/tmp/ipykernel_971747/72093289.py:131: RuntimeWarning: invalid value encountered in true_divide
  combdata_norm = comb_data / np.max(abs(comb_data))


Rupture july6.003126 (3127/3300)
Rupture july6.003127 (3128/3300)
Rupture july6.003128 (3129/3300)
Rupture july6.003129 (3130/3300)
Rupture july6.003130 (3131/3300)
Rupture july6.003131 (3132/3300)
Rupture july6.003132 (3133/3300)
Rupture july6.003133 (3134/3300)
Rupture july6.003134 (3135/3300)
Rupture july6.003135 (3136/3300)
Rupture july6.003136 (3137/3300)
Rupture july6.003137 (3138/3300)
Rupture july6.003138 (3139/3300)
Rupture july6.003139 (3140/3300)
Rupture july6.003140 (3141/3300)
Rupture july6.003141 (3142/3300)
Rupture july6.003142 (3143/3300)
Rupture july6.003143 (3144/3300)
Rupture july6.003144 (3145/3300)
Rupture july6.003145 (3146/3300)
Rupture july6.003146 (3147/3300)
Rupture july6.003147 (3148/3300)
Rupture july6.003148 (3149/3300)
Rupture july6.003149 (3150/3300)
Rupture july6.003150 (3151/3300)
Rupture july6.003151 (3152/3300)
Rupture july6.003152 (3153/3300)
Rupture july6.003153 (3154/3300)
Rupture july6.003154 (3155/3300)
Rupture july6.003155 (3156/3300)
Rupture ju

In [4]:
data_array = np.array(data_list)
print('Data array shape:')
print(data_array.shape) # Arrivals at samples 128, 384, 640
# print(data_array[0])

rupt_array = np.array(rupt_list)
# print(rupt_array.shape)

sta_array = np.array(sta_list)
# print(sta_array.shape)

mag_array = np.array(mag_list)
# print(mag_array.shape)

info_array = np.column_stack((rupt_array, sta_array, mag_array))
print('Info array shape:')
print(info_array.shape)
print(info_array[0])

Data array shape:
(917400, 768)
Info array shape:
(917400, 3)
['july6.000000' 'P595' '7.2110']


In [5]:
h5f = h5py.File('/hdd/rc_fq/summer23/' + project_name + '_128samps_fq_wvfm_data_formatted_normalized.hdf5', 'w')
h5f.create_dataset('data', data = data_array)
h5f.close()

# np.save('/hdd/rc_fq/summer23/' + project_name + '_128samps_fq_wvfm_info.npy', info_array) 